In [96]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import time
import requests
import pandas as pd

In [2]:
user_agent = "Microsoft Edge UA string:Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.10136"
options = webdriver.EdgeOptions()
options.add_argument('user-agent={0}'.format(user_agent))
options.add_argument("--headless=new")

driver = webdriver.Edge()

url = "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_special,video&interests=in0000108"

page = driver.get(url)


button = driver.find_element(By.CSS_SELECTOR, "span.ipc-see-more__text")

actions = ActionChains(driver)
actions.move_to_element(button)
actions.click(button)
actions.perform()

time.sleep(2)   
        
soup = BeautifulSoup(driver.page_source, "html.parser")

urls = []
for i in soup.find_all("li", class_="ipc-metadata-list-summary-item"): 
    urls.append("https://www.imdb.com" + i.find('a', href=True)['href'])
urls[:100]

driver.quit()


In [88]:
def get_movie_info(url): 
    response = requests.get(url, headers=headers)
    print(response.status_code, url)
    user_agent = "Microsoft Edge UA string:Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.10136"
    options = webdriver.EdgeOptions()
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--headless=old")
    driver = webdriver.Edge()
    page = driver.get(url)  
    
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN) 
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN) 
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)  
    
    time.sleep(3)    
    
    soup = BeautifulSoup(driver.page_source, "lxml")
    
    try:
        popularity = soup.find("div", class_="sc-22c33ade-1 cxGAAh").text
        
    except Exception as e: 
        popularity = None


    title = soup.find("span", class_="hero__primary-text").text.strip()
    try:
        rating = soup.find("span", class_= "sc-c4ffe080-1 iQZtLP").text.strip()
        rating = rating[:3]
        
    except Exception as e: 
        rating = None
        
    print(title)
        
    
    try: 
        story = soup.find('div', class_='sc-3c16af05-0 kefoZk')
        storyLine = story.find('div', class_='ipc-html-content-inner-div').text
        
    except Exception as e: 
        storyLine = None
        
    cast_and_crew = soup.find("div", class_="sc-70a366cc-2 bscNnP")
    
    director = cast_and_crew.find_next("a").text
    
    #retreive html to scrape writers
    
    writers = cast_and_crew.li.next_sibling
    
    allWriters = []
    
    for w in writers.find_all("a"):
        allWriters.append(w.text)
        
    if "Writers" in allWriters: 
        allWriters.remove("Writers")
        
    else:  
        pass
    
    writer1 = allWriters[0]
    
    try: 
        writer2 = allWriters[1]
        
    except Exception as e: 
        writer2 = None
        
    #retreive html to scrape stars 
    
    stars = cast_and_crew.li.next_sibling.next_sibling
        
    allStars = []
    
    for s in stars.find_all("a"):
        allStars.append(s.text)
        
    if "Stars" in allStars: 
        allStars.remove("Stars")
        
    else:  
        pass
    
    star1 = allStars[0]
    
    try: 
        star2 = allStars[1]
        
    except Exception as e: 
        star2 = None
        
    movieInfo = soup.find("ul", class_="ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt")
    yearRelease = movieInfo.a.text    
    movieRating = movieInfo.a.find_next("a").text
    runTime = movieInfo.a.find_next("a").find_next("li").text
    
    
    
    return title, rating, story, storyLine, popularity, director, writer1, writer2, star1, star2, yearRelease, movieRating, runTime
    
    

In [73]:
headers =  {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186'}
sleep_min = 0
sleep_max = 1

In [92]:
now = time.time()
titles = []
ratings = []
storyLines = []
popularityScores = []
directors = []
writers1 = []
writers2 = []
stars1 = []
stars2 = []
yearReleases = []
moviesRatings = []
runTimes = []
ids = []
ids_set = set()
sleep_min = 0
sleep_max = 1

for url in urls [:100]:     
    title, rating, story, storyLine, popularity, director, writer1, writer2, star1, star2, yearRelease, movieRating, runTime=get_movie_info(url)
    
    titles.append(title)
    ratings.append(rating)
    storyLines.append(storyLine)
    popularityScores.append(popularity)
    writers1.append(writer1)
    writers2.append(writer2)
    stars1.append(star1)
    stars2.append(star2)
    directors.append(director)
    yearReleases.append(yearRelease)
    moviesRatings.append(movieRating)
    runTimes.append(runTime)
    
print('Took', time.time()-now, 'seconds.')
    
        

200 https://www.imdb.com/title/tt0077651/?ref_=sr_i_1
Halloween
200 https://www.imdb.com/title/tt0072271/?ref_=sr_i_2
The Texas Chain Saw Massacre
200 https://www.imdb.com/title/tt0083907/?ref_=sr_i_3
The Evil Dead
200 https://www.imdb.com/title/tt0100814/?ref_=sr_i_4
Tremors
200 https://www.imdb.com/title/tt0080761/?ref_=sr_i_5
Friday the 13th
200 https://www.imdb.com/title/tt0185937/?ref_=sr_i_6
The Blair Witch Project
200 https://www.imdb.com/title/tt0086320/?ref_=sr_i_7
Sleepaway Camp
200 https://www.imdb.com/title/tt0095444/?ref_=sr_i_8
Killer Klowns from Outer Space
200 https://www.imdb.com/title/tt0063350/?ref_=sr_i_9
Night of the Living Dead
200 https://www.imdb.com/title/tt0082418/?ref_=sr_i_10
Friday the 13th Part 2
200 https://www.imdb.com/title/tt0077681/?ref_=sr_i_11
The Hills Have Eyes
200 https://www.imdb.com/title/tt0068833/?ref_=sr_i_12
The Last House on the Left
200 https://www.imdb.com/title/tt1709143/?ref_=sr_i_13
The Last Days on Mars
200 https://www.imdb.com/title

In [93]:
 spookySZN = pd.DataFrame({'Movie':titles, 
               'Rating':ratings, 
               'Released':yearReleases, 
                'Director':directors,
                'Writer1': writers1,
                'Writer2': writers2, 
                'Star1': stars1, 
                'Star2': stars2, 
               'MovieRating':moviesRatings, 
               'RunTime':runTimes, 
                'Synopsis':storyLines, 
                'PopularityScore':popularityScores
              }
             )

In [94]:
spookySZN

,Movie,Rating,Released,Director,Writer1,Writer2,Star1,Star2,MovieRating,RunTime,Synopsis,PopularityScore
0,Halloween,7.7,1978,John Carpenter,John Carpenter,Debra Hill,Donald Pleasence,Jamie Lee Curtis,R,1h 31m,None,288
1,The Texas Chain Saw Massacre,7.4,1974,Tobe Hooper,Kim Henkel,Tobe Hooper,Marilyn Burns,Edwin Neal,R,1h 23m,None,447
2,The Evil Dead,7.4,1981,Sam Raimi,Sam Raimi,None,Bruce Campbell,Ellen Sandweiss,NC-17,1h 25m,Five college students take time off to spend a...,652
3,Tremors,7.2,1990,Ron Underwood,S.S. Wilson,Brent Maddock,Kevin Bacon,Fred Ward,PG-13,1h 36m,A small town gradually becomes aware of a stra...,711
4,Friday the 13th,6.4,1980,Sean S. Cunningham,Victor Miller,Ron Kurz,Betsy Palmer,Adrienne King,R,1h 35m,"In 1957, a young boy named Jason drowns in a l...",787
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Reptilicus,3.6,1961,Poul Bang,Sidney W. Pink,Ib Melchior,Bent Mejding,Asbjørn Andersen,Approved,1h 22m,A portion of the tail of a giant prehistoric r...,None
96,Modern Vampires,4.6,1998,Richard Elfman,Matthew Bright,None,Casper Van Dien,Natasha Gregson Wagner,R,1h 35m,"Count Dracula is in L.A., running a hideous cl...",None
97,Night of the Ghouls,3.6,1959,Edward D. Wood Jr.,Edward D. Wood Jr.,None,Kenne Duncan,Duke Moore,Approved,1h 9m,"Follow-up to Ed Wood's ""Plan 9 from Outer Spac...",None
98,Hell's Trap,5.6,1989,Pedro Galindo III,Pedro Galindo III,Santiago Galindo,Pedro Fernández,Edith González,Unrated,1h 30m,Seven young people go to a desolated forest lo...,None


In [95]:
spookySZN.to_csv('spookySZN')